In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv(r'C:\Users\DELL\OneDrive\Desktop\Data Science\DS INTERN\mlflow experiemnt tracking and model management\Data\reviews_badminton\data.csv')

In [3]:
df

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1
...,...,...,...,...,...,...,...,...
8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
8514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
8516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [4]:
df.isnull().sum()

Reviewer Name       10
Review Title        10
Place of Review     50
Up Votes            10
Down Votes          10
Month              465
Review text          8
Ratings              0
dtype: int64

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.dropna(subset=['Review text'], inplace=True)

In [7]:
df.isna().sum()

Reviewer Name        2
Review Title         2
Place of Review     42
Up Votes             2
Down Votes           2
Month              457
Review text          0
Ratings              0
dtype: int64

In [8]:
df=df[df['Ratings']!=3]

In [9]:
sentiment={
    1:0,
    2:0,
    4:1,
    5:1
}

In [10]:
df.loc[:,'Sentiment'] = df['Ratings'].map(sentiment)

In [11]:
df.loc[:,'Review text'] = df['Review text'].str.replace('READ MORE', '')

In [12]:
df


,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings,Sentiment
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4,1
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1,0
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1,0
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1,0
5,Baji Sankar,Mind-blowing purchase,"Certified Buyer, Hyderabad",173.0,45.0,Oct 2018,Good quality product. Delivered on time.,5,1
...,...,...,...,...,...,...,...,...,...
8504,naresh g,For Mavis350,NaN,2.0,1.0,Aug 2016,Received product intact and sealed,5,1
8506,Abani Behera,Don't waste your money,NaN,0.0,2.0,Sep 2016,up to the mark but same is available in market...,4,1
8507,vishnu varma,Really Nice,"Certified Buyer, Agartala",0.0,1.0,Sep 2016,Nice delivery speed,5,1
8508,NaN,NaN,NaN,NaN,NaN,NaN,No complaints about the item . Its the best on...,5,1


In [13]:
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [14]:
def preprocess(text, flag, n=2):
    # remove special characters
    sentence = re.sub(r'[^a-zA-Z]', ' ', text)
    # convert sentence into lower case
    sentence = sentence.lower()
    # tokenize sentence into words
    tokens = sentence.split()
    
    # Remove 'not' from default English stopwords
    custom_stopwords = set(stopwords.words("english")) - {'not'}
    
    # remove stop words
    clean_tokens = [token for token in tokens if token not in custom_stopwords]
    
    # stemming/lemmatization
    if flag == 'stem':
        clean_tokens = [stemmer.stem(token) for token in clean_tokens]
    else:
        clean_tokens = [lemmatizer.lemmatize(token) for token in clean_tokens]
    
    # Generate n-grams
    ngrams_list = list(ngrams(clean_tokens, n))
    ngrams_text = [' '.join(gram) for gram in ngrams_list]
    
    return pd.Series(' '.join(ngrams_text))

In [15]:
df['Review text'] = df['Review text'].apply(lambda x: preprocess(x, flag='lemmatize', n=2))

In [16]:
vector = TfidfVectorizer()
vector.fit_transform(df['Review text']).toarray()
text = "".join(df['Review text'].values.tolist())

In [17]:
X=df['Review text']
y=df['Sentiment']

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=2)

In [19]:
import mlflow

In [20]:
mlflow.set_experiment("sentiment_analysis")

2024/04/05 18:44:45 INFO mlflow.tracking.fluent: Experiment with name 'sentiment_analysis' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/DELL/OneDrive/Desktop/Data%20Science/DS%20INTERN/mlflow%20experiemnt%20tracking%20and%20model%20management/mlruns/803727640447532239', creation_time=1712322885589, experiment_id='803727640447532239', last_update_time=1712322885589, lifecycle_stage='active', name='sentiment_analysis', tags={}>

In [21]:


# Define the pipeline
rf_pipe_tfidf = Pipeline([
    ('Vectorization', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])

# Define the parameter grid for GridSearchCV
rf_param_grid_tfidf = {
    'Vectorization': [TfidfVectorizer()],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
rf_clf_tfidf = GridSearchCV(
    estimator=rf_pipe_tfidf, 
    param_grid=rf_param_grid_tfidf, 
    scoring='accuracy',
    cv=5,
    return_train_score=True,
    verbose=1
)

# Enable auto-logging with MLflow
mlflow.sklearn.autolog()

# Start an MLflow run
with mlflow.start_run() as run:
    # Fit the GridSearchCV object
    rf_clf_tfidf.fit(X_train, y_train)


2024/04/05 18:45:38 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/04/05 18:45:39 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 27 candidates, totalling 135 fits


2024/04/05 18:52:24 INFO mlflow.sklearn.utils: Logging the 5 best runs, 22 runs will be omitted.


In [22]:
from sklearn.tree import DecisionTreeClassifier

# Define the pipeline
dt_pipe_tfidf = Pipeline([
    ('Vectorization', TfidfVectorizer()),
    ('classifier', DecisionTreeClassifier())
])

# Define the parameter grid for GridSearchCV
dt_param_grid_tfidf = {
    'Vectorization': [TfidfVectorizer()],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
dt_clf_tfidf = GridSearchCV(
    estimator=dt_pipe_tfidf, 
    param_grid=dt_param_grid_tfidf, 
    scoring='accuracy',
    cv=5,
    return_train_score=True,
    verbose=1
)

# Enable auto-logging with MLflow
mlflow.sklearn.autolog()

# Start an MLflow run
with mlflow.start_run() as run:
    # Fit the GridSearchCV object
    dt_clf_tfidf.fit(X_train, y_train)


2024/04/05 18:54:05 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


2024/04/05 18:54:39 INFO mlflow.sklearn.utils: Logging the 5 best runs, 4 runs will be omitted.


In [23]:
pipelines = {
    'RandomForest': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', RandomForestClassifier())
    ]),
    'DecisionTrees': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', DecisionTreeClassifier())
    ]),
    'LogisticRegression': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', LogisticRegression())
    ]),
    'KNeighbors': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', KNeighborsClassifier())
    ]),
    'NaiveBayes': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', MultinomialNB())
    ]),
    'SVC': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', SVC())
    ]),
    'XGBoost': Pipeline([
        ('Vectorization', TfidfVectorizer()),
        ('classifier', XGBClassifier())
    ])
    
}

param_grids = {
    'RandomForest': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5, 10]
    },
    'DecisionTrees': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5, 10]
    },
    'LogisticRegression': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2']
    },
    'KNeighbors': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__n_neighbors': [3, 5, 7, 9],
        'classifier__p': [1, 2, 3]
    },
    'NaiveBayes': {
        'Vectorization': [TfidfVectorizer()],
    },
    'SVC': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    },
    'XGBoost': {
        'Vectorization': [TfidfVectorizer()],
        'classifier__learning_rate': [0.01, 0.1, 0.3],
        'classifier__max_depth': [3, 5, 7],
        'classifier__n_estimators': [50, 100, 200]
    }
     
}


In [24]:
best_models = {}

# Run the Pipeline
for algo in pipelines.keys():
    print("*"*10, algo, "*"*10)
    grid_search = GridSearchCV(estimator=pipelines[algo], 
                               param_grid=param_grids[algo], 
                               cv=5, 
                               scoring='accuracy', 
                               return_train_score=True,
                               verbose=1
                              )
    
    mlflow.sklearn.autolog(max_tuning_runs=None)
    
    with mlflow.start_run() as run:
        %time grid_search.fit(X_train, y_train)
        
    print('Train Score: ', grid_search.best_score_)
    print('Test Score: ', grid_search.score(X_test, y_test))
    
    best_models[algo] = grid_search.best_estimator_
    print()

********** RandomForest **********


2024/04/05 19:02:39 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 27 candidates, totalling 135 fits
CPU times: total: 5min 57s
Wall time: 6min 41s
Train Score:  0.9383540027369142
Test Score:  0.9397163120567376

********** DecisionTrees **********


2024/04/05 19:09:21 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
CPU times: total: 21.8 s
Wall time: 41.8 s
Train Score:  0.9341323126924393
Test Score:  0.9376899696048632

********** LogisticRegression **********


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
CPU times: total: 2.23 s
Wall time: 14.9 s
Train Score:  0.8800878093283157
Test Score:  0.894630192502533

********** SVC **********


2024/04/05 19:11:56 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 6 candidates, totalling 30 fits
CPU times: total: 39.8 s
Wall time: 58.1 s
Train Score:  0.9334564944691527
Test Score:  0.939209726443769

********** XGBoost **********


2024/04/05 19:12:55 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 27 candidates, totalling 135 fits
CPU times: total: 9min 59s
Wall time: 2min 7s
Train Score:  0.9383544303797468
Test Score:  0.9422492401215805

